<a href="https://colab.research.google.com/github/Coolicc/Big-Data-Analysis-PySpark/blob/main/Ognjen_Kulic_4_%C4%8Das.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####Podešavanje Sparka za rad u Google Colabu
#####Izvršite sve komande u datom redosledu

In [ ]:
!java -version

openjdk version "11.0.6" 2020-01-14
OpenJDK Runtime Environment (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1)
OpenJDK 64-Bit Server VM (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1, mixed mode, sharing)


In [ ]:
!update-alternatives --config java

There are 2 choices for the alternative java (providing /usr/bin/java).

  Selection    Path                                            Priority   Status
------------------------------------------------------------
* 0            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      auto mode
  1            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      manual mode
  2            /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java   1081      manual mode

Press <enter> to keep the current choice[*], or type selection number: 


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
!pip install pyspark

     |████████████████████████████████| 217.8MB 62kB/s 
     |████████████████████████████████| 204kB 42.1MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=7e65cbeaefb02827f973fc292a4b920cc8a7e38d8cd0f6dcca1de702389585ee
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


####Dokumentacija za funkcije u PySpark-u https://spark.apache.org/docs/latest/api/python/index.html

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


###Zadatak 1
Preuzeti fajl sa adrese https://drive.google.com/file/d/1CYwx-T0TtjF_MfWIkYiEjirKK6LjS3pz/view?usp=sharing i dodati ga na vaš Google Drive. Separtor je znak ~. Fajl predstavlja kolekciju postova. Kolone su redom: broj komentara na postu, datum poslednje aktivnosti, id vlasnika posta, sadržaj posta, ocena, datum kreiranja, broj pregleda,naslov, tagovi, broj odgovora, id prihvaćenog odgovora,tip posta (1 ako je pitanje, 2 ako je odgovor), id posta. Odgovori na post su takođe postovi pa postoji zavisnost između id odgovora i samog id posta (zamislite rekurzivnu relaciju u relacionoj bazi)

Inicijalizovati Spark Session

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()


Fajl nema header zato ćemo kreirati semu gde ćemo definisati nazive kolona i njihove tipove. Ukoliko to ne uradimo sve kolone će biti tipa String

In [ ]:
#ovo je šema za zadati fajl
from pyspark.sql.types import *
from pyspark.sql import Row
from datetime import datetime

postSchema = StructType([
  StructField("commentCount", IntegerType(), True),
  StructField("lastActivityDate", TimestampType(), True),
  StructField("ownerUserId", IntegerType(), True),
  StructField("body", StringType(), True),
  StructField("score", IntegerType(), True),
  StructField("creationDate", TimestampType(), True),
  StructField("viewCount", IntegerType(), True),
  StructField("title", StringType(), True),
  StructField("tags", StringType(), True),
  StructField("answerCount", IntegerType(), True),
  StructField("acceptedAnswerId", LongType(), True),
  StructField("postTypeId", IntegerType(), True),
  StructField("id", IntegerType(), False)
  ])

Ucitati fajl italianPosts.csv i definisati mu šemu

In [ ]:
#pri testiranju promeniti putanju do fajla u skladu sa vasom putanjom
postsDF = spark.read.csv("/content/drive/My Drive/italianPosts.csv",sep='~', schema=postSchema)

In [ ]:
postsDF.show(30)

+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+
|commentCount|    lastActivityDate|ownerUserId|                body|score|        creationDate|viewCount|               title|                tags|answerCount|acceptedAnswerId|postTypeId|  id|
+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+
|        null|                null|       null|                null| null|                null|     null|                null|                null|       null|            null|      null|null|
|        null|                null|       null|                null| null|                null|     null|                null|                null|       null|            null|      null|null|
|        null|                null|

###Zadatak 1
Prikazati samo one vrste koje u koloni body sadrže reč Italiano

In [ ]:
italianoDF = postsDF.filter(postsDF.body.contains('Italiano') | postsDF.body.contains('italiano'))
print(italianoDF.count())
print(italianoDF.show(italianoDF.count()))
print('--------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
print(italianoDF.select('body').count())
print(italianoDF.select('body').show(italianoDF.select('body').count()))

145
+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+
|commentCount|    lastActivityDate|ownerUserId|                body|score|        creationDate|viewCount|               title|                tags|answerCount|acceptedAnswerId|postTypeId|  id|
+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+
|           2|2013-11-11 23:23:...|        159|&lt;p&gt;Sono un'...|    7|2013-11-11 18:19:...|      138|origine dell'espr...|&lt;idioms&gt;&lt...|          2|            1185|         1|1184|
|           2|2013-11-27 14:21:...|         22|&lt;p&gt;Ho solo ...|    3|2013-11-23 09:15:...|      474|Uso nell'italiano...|&lt;word-choice&g...|          1|            1408|         1|1366|
|           3|2013-11-27 12:03:

###Zadatak 2
Prebrojati koliko postova koji su pitanje nemaju prihvaćen odgovor

In [ ]:
postsDF.filter('(postTypeId == 1) and (acceptedAnswerId is null)').count()

0

###Zadatak 3
Promeniti naziv kolone ownerUserId u owner

In [ ]:
postsDF2 = postsDF.withColumnRenamed("ownerUserId", "owner")
postsDF2.show(5)

+------------+--------------------+-----+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+
|commentCount|    lastActivityDate|owner|                body|score|        creationDate|viewCount|               title|                tags|answerCount|acceptedAnswerId|postTypeId|  id|
+------------+--------------------+-----+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+
|        null|                null| null|                null| null|                null|     null|                null|                null|       null|            null|      null|null|
|        null|                null| null|                null| null|                null|     null|                null|                null|       null|            null|      null|null|
|        null|                null| null|                null| nu

###Zadatak 4
Dodati kolonu "ratio" koja sadrži odnos viewsCount/score

In [ ]:
ratioDF = postsDF2.withColumn("ratio", postsDF2.viewCount / postsDF2.score)
ratioDF.show()

+------------+--------------------+-----+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+------------------+
|commentCount|    lastActivityDate|owner|                body|score|        creationDate|viewCount|               title|                tags|answerCount|acceptedAnswerId|postTypeId|  id|             ratio|
+------------+--------------------+-----+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+------------------+
|        null|                null| null|                null| null|                null|     null|                null|                null|       null|            null|      null|null|              null|
|        null|                null| null|                null| null|                null|     null|                null|                null|       null|            null|      

###Zadatak 5
Selektovati 10 pitanja koja su poslednja azurirana

In [ ]:
ratioDF.sort(ratioDF.lastActivityDate.desc()).limit(10).show()

+------------+--------------------+-----+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+-----+
|commentCount|    lastActivityDate|owner|                body|score|        creationDate|viewCount|               title|                tags|answerCount|acceptedAnswerId|postTypeId|  id|ratio|
+------------+--------------------+-----+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+-----+
|           2|2014-09-13 22:25:...|  707|&lt;p&gt;Ho osser...|    3|2014-09-12 09:44:...|      102|Perché a volte si...|&lt;orthography&g...|          1|            2344|         1|2343| 34.0|
|           0|2014-09-12 10:55:...|    8|&lt;p&gt;Wikipedi...|   10|2013-11-20 16:42:...|      145|Plural form of &q...|&lt;orthography&g...|          5|            1336|         1|1321| 14.5|
|           1|2014-09-12 07:32:...|

###Zadatak 6
Pronaći post koji je tipa pitanje i bio je najduže aktivan (najveća je razlika između datuma kreiranja i vremena poslednje aktivnosti)

In [ ]:
import pyspark.sql.functions as f
activeDF = ratioDF.filter(ratioDF.postTypeId == 1).withColumn('activeFor', f.datediff(ratioDF.lastActivityDate, ratioDF.creationDate))
activeDF.sort(activeDF.activeFor.desc()).show(1)

+------------+--------------------+-----+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+-----+---------+
|commentCount|    lastActivityDate|owner|                body|score|        creationDate|viewCount|               title|                tags|answerCount|acceptedAnswerId|postTypeId|  id|ratio|activeFor|
+------------+--------------------+-----+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+-----+---------+
|           3|2014-09-11 14:37:...|   63|&lt;p&gt;The plur...|    5|2013-11-12 13:34:...|       59|Why is the plural...|&lt;plural&gt;&lt...|          1|            1227|         1|1221| 11.8|      303|
+------------+--------------------+-----+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+--

###Zadatak 7
Prikazati prosečan i maksimalna skor za postove koji su tipa pitanje

In [ ]:
qDF = ratioDF.filter(ratioDF.postTypeId == 1)
print(qDF.agg(f.avg(qDF.score)).collect())
print(qDF.agg(f.max(qDF.score)).collect())

[Row(avg(score)=4.735294117647059)]
[Row(max(score)=20)]


###Zadatak 8
popuniti u koloni viewCount null vrednost sa vrednošću 0

In [ ]:
vcDF = ratioDF.fillna({'viewCount': 0})
vcDF.show()

+------------+--------------------+-----+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+------------------+
|commentCount|    lastActivityDate|owner|                body|score|        creationDate|viewCount|               title|                tags|answerCount|acceptedAnswerId|postTypeId|  id|             ratio|
+------------+--------------------+-----+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+------------------+
|        null|                null| null|                null| null|                null|        0|                null|                null|       null|            null|      null|null|              null|
|        null|                null| null|                null| null|                null|        0|                null|                null|       null|            null|      

###Zadatak 9
Prikazati ukupan broj postova za svakog autora

In [ ]:
ratioDF.filter(ratioDF.owner.isNotNull()).groupBy('owner').count().collect()

[Row(owner=833, count=3),
 Row(owner=623, count=1),
 Row(owner=516, count=1),
 Row(owner=85, count=2),
 Row(owner=808, count=12),
 Row(owner=580, count=2),
 Row(owner=53, count=2),
 Row(owner=804, count=1),
 Row(owner=799, count=2),
 Row(owner=133, count=1),
 Row(owner=633, count=1),
 Row(owner=375, count=4),
 Row(owner=683, count=1),
 Row(owner=744, count=1),
 Row(owner=193, count=90),
 Row(owner=796, count=1),
 Row(owner=497, count=7),
 Row(owner=300, count=2),
 Row(owner=842, count=1),
 Row(owner=501, count=1),
 Row(owner=626, count=1),
 Row(owner=159, count=5),
 Row(owner=606, count=8),
 Row(owner=806, count=1),
 Row(owner=103, count=1),
 Row(owner=12, count=109),
 Row(owner=223, count=14),
 Row(owner=388, count=1),
 Row(owner=91, count=1),
 Row(owner=663, count=1),
 Row(owner=707, count=40),
 Row(owner=22, count=44),
 Row(owner=685, count=1),
 Row(owner=230, count=3),
 Row(owner=190, count=6),
 Row(owner=232, count=1),
 Row(owner=360, count=1),
 Row(owner=725, count=8),
 Row(owner